# RNN with LSTM for Predictive Traffic

In [62]:
import time
import math
import random

import os
import glob
import csv
import datetime

import numpy as np
import torch
import torch.nn as nn

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Preparing the data

In [82]:
# CSV file
file = '210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv'

# Weather data at a postcode
class WeatherData:
    # Weather data
    time = []
    hour = []
    weekday = []
    week_number = []

    temperature = []
    wind_speed = []
    precipitation = []
    radiation = []

    def __init__(self, postcode):
        self.postcode = postcode

    # Adds weather data to the postcode
    def add_data(self, time, temperature, wind_speed, precipitation, radiation):
        self.time.append(time)
        self.hour.append(time.hour)
        self.weekday.append(time.weekday())
        self.week_number.append(time.isocalendar()[1])

        self.temperature.append(temperature)
        self.wind_speed.append(wind_speed)
        self.precipitation.append(precipitation)
        self.radiation.append(radiation)


# Weather for each postcode
weather = {}

# Open the CSV file
with open(file) as csv_file:
    # Read the CSV file with delimiter ';'
    csv_reader = csv.reader(csv_file, delimiter = ";")
    i = 0

    # Iterate through CSV rows
    for row in csv_reader:
        # Skip first 2 lines
        if i < 2:
            i = i + 1
            continue

        # Get parameters
        time = datetime.datetime.strptime(row[0][:-3], '%d.%m.%Y %H')
        postcode = int(row[1])
        temperature = float(row[2].replace(",", "."))

        if row[3] == '':
            wind_speed = 0
        else:
            wind_speed = float(row[3].replace(",", "."))

        precipitation = float(row[4].replace(",", "."))
        radiation = float(row[5].replace(",", "."))

        # Get current weather instance
        if (postcode in weather) == False:
            weather[postcode] = WeatherData(postcode)

        current = weather[postcode]

        # Add wheather data
        current.add_data(time, temperature, wind_speed, precipitation, radiation)

data = weather[76131]
#array = np.array([data.temperature, data.wind_speed, data.precipitation, data.radiation, data.hour, data.weekday, data.week_number])

n_samples = len(data.temperature)

def random_choice(a):
    # get random index from list
    random_idx = random.randint(0, a-1)
    return random_idx

def weather_to_tensor(data=data, batch=2, input_size=7):
    
    """
    weather_to_tensor() -> Tensor

    Returns a random tensor of shape (seq_len, batch, input_size)

    Keywords:
        data - class whitch provides weather data
        batch - total number of training examples present in a single batch
        input_size - the number of expected features in the input x   
    """

    tensor = torch.zeros(1, batch, input_size)

    for i in range(batch):
        sample_index = random_choice(n_samples)
        #print(sample_index)

        tensor[0][i][0] = data.temperature[sample_index]
        tensor[0][i][1] = data.wind_speed[sample_index]
        tensor[0][i][2] = data.precipitation[sample_index]
        tensor[0][i][3] = data.radiation[sample_index]
        tensor[0][i][4] = data.hour[sample_index]
        tensor[0][i][5] = data.weekday[sample_index]
        tensor[0][i][6] = data.week_number[sample_index]

    return tensor

x = weather_to_tensor()
print(x)

traffic_tensor = torch.tensor([3], dtype=torch.long)

print(trafic_tensor.item())

2019-11-20 18:00:00
tensor([[[ 19.7693,   2.3413,   0.0000, 609.8613,   8.0000,   2.0000,  31.0000],
         [ 21.9312,   6.0098,   0.0000,   0.0000,  19.0000,   1.0000,  39.0000]]])
3


## Defining network structure

In [69]:
class RNN(nn.Module):

    def __init__(self, input_size, hidden_size, n_layers, n_traffic):
        super(RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, n_layers, batch_first=False)
        # -> x of shape (seq_len, batch, hidden_size)
        self.fc = nn.Linear(hidden_size, n_traffic)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # initial hidden- & cell-state
        h0 = torch.zeros(self.n_layers, x.size(1), self.hidden_size).to(device)
        c0 = torch.zeros(self.n_layers, x.size(1), self.hidden_size).to(device)

        # output of shape (seq_len, batch, n_directions * hidden_size)
        out, _ = self.lstm(x, (h0,c0))
        
        # only last Time-Step
        out = out[-1, :, :]
        
        out = self.fc(out)
        out = self.softmax(out)

        return out

## Defining helper functions for neural networks's training

In [65]:
def train(weather_tensor, traffic_tensor):
    output = model(weather_tensor)
    loss = criterion(output, traffic_tensor)
    # traffic_tensor -> 
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return output, loss.item()

def traffic_from_output(output):
    traffic_idx = torch.argmax(output).item()
    return traffic_idx

## Defining hyperparameters of neural network's training

In [70]:
# hyper parameters
input_size = 7 # features
n_traffic = 10 # output_size
hidden_size = 128
n_layers = 1

learning_rate = 0.005

## Start training loop

In [78]:
# initialize neural network   
model = RNN(input_size, hidden_size, n_layers, n_traffic).to(device)

# loss and optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

current_loss = 0
all_losses = []

print_every = 1
plot_every = 1000
n_iters = 50000

#def timeSince(since):
    #now = time.time()
    #s = now - since
    #m = math.floor(s / 60)
    #s -= m * 60
    #return '%dm %ds' % (m, s)

#start = time.time()

for i in range(1, n_iters + 1):
    #character, file, character_tensor, file_tensor = random_training_example(character_files, all_characters)
    weather_tensor = weather_to_tensor(data, 1, 7)
    output, loss = train(weather_tensor.to(device), traffic_tensor.to(device))
    current_loss += loss
    
    # print iter number, loss, name and guess
    if i % print_every == 0:
        guess = traffic_from_output(output)
        correct = '✓' if guess == trafic_tensor.item() else '✗ (%s)' % trafic_tensor.item()
        #print('%d %d%% (%s) %.4f %s / %s %s' % (i, i / n_iters * 100, timeSince(start), loss, file, guess, correct))
        print('%d %d%% %.4f %s / %s %s' % (i, i / n_iters * 100, loss, file, guess, correct))

    # add current loss avg to list of losses
    if i % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

1 0% 1.7303 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 4 ✗ (3)
2 0% 1.5335 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 4 ✗ (3)
3 0% 1.5714 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 4 ✗ (3)
4 0% 1.7771 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 2 ✗ (3)
5 0% 1.3935 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
6 0% 1.4239 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
7 0% 1.1943 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
8 0% 1.0050 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
9 0% 1.3314 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
10 0% 1.1780 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
11 0% 1.2046 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
12 0% 1.3707 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
13 0% 1.1199 210226_Daten_Hackathon/210226_Daten_Wetter_Hackathon.csv / 3 ✓
14 0%

KeyboardInterrupt: 

## Plot LOSS-function

In [ ]:
# create figure
mpl.style.use("seaborn-whitegrid")
plt.figure(figsize=(12,6))
# create plot
plt.plot(all_losses)
# title and labels
plt.title("LOSS-function", fontsize=20)
plt.xlabel("iterations", fontsize=15)
plt.ylabel("loss", fontsize=15)
# show plot
plt.show()

## Save model

In [5]:
FILE = "model.pth"
torch.save(model.state_dict(), FILE)

## Load Model

In [ ]:
FILE = "model.pth"
loaded_model = RNN(input_size, hidden_size, n_layers, n_characters).to(device)
loaded_model.load_state_dict(torch.load(FILE, map_location=device))
loaded_model.eval()

## Determin accuracy

In [ ]:
# load test dataset
character_files, all_characters = load_data(subfolder="test")
print("characters in test data: ", all_characters)

# keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_characters, n_characters)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for character in all_characters:
        # all files from one character
        for file in character_files[character]:
            file_tensor = file_to_tensor(file).to(device)
            # get output
            output = loaded_model(file_tensor).to(device)
            # get character from output
            guess = character_from_output(output)
            n_samples += 1
            if guess == character:
                n_correct += 1
                correct = "✓"
            else:
                correct = "✗ (%s)" % character
            confusion[all_characters.index(character)][all_characters.index(guess)] += 1
            
            print(file, "/", guess, correct)

# normalize by dividing every row by its sum
for i in range(n_characters):
    confusion[i] = confusion[i] / confusion[i].sum()

acc = 100.0 * n_correct / n_samples
print(f"\naccuracy = {acc:.2f} %\n")

# set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# set up axes
ax.set_xticklabels([''] + all_characters, rotation=90)
ax.set_yticklabels([''] + all_characters)

# force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()